In [11]:
# Directory of stored data
filedir: str = "C:\\Users\\alvar\\University of Southampton\\GDP - General\\Flight Day 2\\data"

# Flight number
flight: float = 2

In [12]:
import pandas as pd
from datetime import datetime
import re

In [13]:
def regexdatetime(input_comment: str) -> datetime:
    result = re.search(r"t(-?[0-9]+.[0-9]+)c?(-?[0-9]+)?", input_comment)
    if result.group(2) is not None:
        return datetime.fromtimestamp(float(result.group(2))/1e6)
    return datetime.fromtimestamp(float(result.group(1)))

def regextimestamp(input_comment: str) -> float:
    result = re.search(r"t(-?[0-9]+.[0-9]+)c?(-?[0-9]+)?", input_comment)
    if result.group(2) is not None:
        return float(result.group(2))/1e6
    return float(result.group(1))

def load(dir: str, ATOT: datetime, ALDT: datetime, flight: int) -> pd.DataFrame:
    data = pd.read_csv(dir + "\\ps_data.csv", index_col=False)
    data["time"] = data["comment"].map(regexdatetime)
    data = data[(data["time"] > ATOT) & (data["time"] < ALDT)]
    data["timestamp"] = data["comment"].map(regextimestamp)

    # TODO: Apply time offset for non calibrated

    arsp = pd.read_csv(dir + f"\\arsp{flight}.csv")
    imu = pd.read_csv(dir + f"\\imu{flight}.csv")
    ahr = pd.read_csv(dir + f"\\ahr2{flight}.csv")
    aoa = pd.read_csv(dir + f"\\aoa{flight}.csv")

    data = pd.merge_asof(data, arsp, on='timestamp')
    data = pd.merge_asof(data, imu, on='timestamp')
    data = pd.merge_asof(data, ahr, on='timestamp')
    data = pd.merge_asof(data, aoa, on='timestamp')

    data = data[data['Airspeed'].notna()]

    return data.reset_index()

if flight == 1:
    data = load(filedir, datetime(2023, 5, 2, 11, 0, 0), datetime(2023, 5, 2, 12, 0, 0), flight)
elif flight == 2:
    data = load(filedir, datetime(2023, 5, 2, 13, 0, 0), datetime(2023, 5, 2, 14, 0, 0), flight)

C:\Users\alvar\AppData\Local\Temp\ipykernel_7216\1250394461.py:29: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TimeUS_x'} in the result is deprecated and will raise a MergeError in a future version.
  data = pd.merge_asof(data, aoa, on='timestamp')
